In [1]:
import time
start_time = time.time()
import cv2
import numpy as np
import math
from PIL import Image
%matplotlib auto
image = cv2.imread('bird.jpg',1)
cv2.imshow('original', image)

Using matplotlib backend: <object object at 0x0000027100494090>


In [2]:
img_str = cv2.imencode('.jpg', image)[1].tobytes()
if len(img_str)%8 != 0:#check for compatibility
    while len(img_str)%8 != 0:
        img_str += b'\x00'

og_img_np = np.frombuffer(img_str, dtype=np.uint8)

In [3]:
import blowfish
cipher = blowfish.Cipher(b'cipherlendontmatter')

In [4]:
data = img_str # electronic codebook mode

data_encrypted = b"".join(cipher.encrypt_ecb(data))

#assert data == data_decrypted

In [5]:
from SystemParams import *

#KGC generates params
kgc_1 = KGC(256)
params = Params(kgc_1.p, kgc_1.q, kgc_1.g, kgc_1.p_pub)  # params generated by KGC
Alice = User( 'Alice', params )
Bob = User( 'Bob', params )
kgc_1.partialkey_compute(Alice)
kgc_1.partialkey_compute(Bob)

In [6]:
m = int.from_bytes(data_encrypted, "big")
a = m
# break into list if it is too long
m_list = []
while m > 0:
    m_list.append(m % 10000000000)
    m = m // 10000000000

#signcrypt every item in the list
Signcryption_text = []
for i in range(len(m_list)):
    Signcryption_text.append(Alice.signcrypt( Bob, params, m_list[i] ))

In [7]:
#unsigncrypt every item in the list
m_un = []
for i in range(len(Signcryption_text)):
    m_un.append(Bob.unsigncrypt( Alice, params, Signcryption_text[i] ))

#merge list back into one number
m_un.reverse()
m_un_merge = 0
for i in range(len(m_un)):
    m_un_merge = m_un_merge * 10000000000 + m_un[i]

m_un_merge2 = int(m_un_merge)

#convert number back to bytes
m_un_merge_bytes = m_un_merge2.to_bytes(len(data_encrypted), "big")

In [8]:
data_decrypted = b"".join(cipher.decrypt_ecb(m_un_merge_bytes))

In [9]:
img_str_np = np.frombuffer(data_decrypted, dtype=np.uint8)
img = cv2.imdecode(img_str_np, 1)

In [10]:
cv2.imshow('decrypted', img)

In [11]:
#img_str2 = cv2.imencode('.jpg', img)[1].tobytes()
#img_str2 = img_str2[0:len(img_str)]
#img_str_np2 = np.frombuffer(img_str2, dtype=np.uint8)

In [12]:
def nearest_square(limit):
    next = math.floor(math.sqrt(limit)) + 1
    return next * next

In [13]:
img_str_np_enc = np.frombuffer(data_encrypted, dtype=np.uint8)
#add elements to the end of the array to make it a square
q = nearest_square(len(img_str_np_enc))

while len(img_str_np_enc) != q:
    img_str_np_enc = np.append(img_str_np_enc, 0)
    

img_str_np_enc1 = img_str_np_enc.reshape(int(math.sqrt(len(img_str_np_enc))), int(math.sqrt(len(img_str_np_enc))))

In [14]:
enc_img = Image.fromarray(img_str_np_enc1)
enc_img.show()


In [15]:
def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(255 / math.sqrt(mse))

d = psnr(image, img)
print(d)

44.784548468233524


In [16]:
cv2.PSNR(image, img)

44.07840825794602

In [17]:
MSE = np.square(np.subtract(img_str_np,og_img_np)).mean()
MSE

0.0

In [18]:
Y = np.square(np.subtract(image,img)).mean()
print("MSE:", Y)

MSE: 2.160854446100348


In [19]:
from skimage.metrics import structural_similarity as ssim

if image is None or img is None:
    print("Error: One or both image files could not be loaded.")
else:
    # Convert the images to grayscale
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Compute SSIM between the two images
    ssim_index, ssim_image = ssim(image_gray, img_gray, full=True)

    print(f'SSIM index: {ssim_index}')

SSIM index: 0.9995827122453412


In [20]:
def jpeg_compression(image, quality, compressed_path):
    # Save the image with JPEG compression
    cv2.imwrite(compressed_path, image, [int(cv2.IMWRITE_JPEG_QUALITY), quality])

if image is None or img is None:
    print("Error: One or both image files could not be loaded.")
else:
    # Path to save the compressed image
    compressed_path = 'compressed_image.jpg'

    # Perform JPEG compression on the second image (img) with a quality factor (e.g., 50)
    jpeg_compression(img, 50, compressed_path)

    # Load the compressed image
    compressed_image = cv2.imread(compressed_path, cv2.IMREAD_COLOR)

    # Convert the original image (image) and compressed image to grayscale
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    compressed_image_gray = cv2.cvtColor(compressed_image, cv2.COLOR_BGR2GRAY)

    # Compute SSIM between the original image (image) and the compressed image
    ssim_index, ssim_image = ssim(image_gray, compressed_image_gray, full=True)

    print(f'SSIM index between original image and compressed image: {ssim_index}')

SSIM index between original image and compressed image: 0.9771716491459778


In [21]:
np.corrcoef(image.ravel(), img.ravel())

array([[1.        , 0.99891476],
       [0.99891476, 1.        ]])

In [22]:
def entropy(array):
    total_entropy = 0
    for i in array:
        total_entropy += -i * math.log(i, 2)
    return total_entropy

In [23]:
image = image.ravel()
img = img.ravel()
for i in range(len(image)) :
    if image[i] == 0 :
        image[i] = 1
    if img[i] == 0 :
        img[i] = 1

out = np.divide(img, image)
a = np.sum(out)
for i in range(len(out)) :
    out[i] = out[i] / a

entropy(out)

17.181638651449212

In [24]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 2238.033324956894 seconds ---


In [25]:
entropy(img.ravel())

C:\Users\Shriansh\AppData\Local\Temp\ipykernel_7048\3076193629.py:4: RuntimeWarning: overflow encountered in scalar negative
  total_entropy += -i * math.log(i, 2)


165149154.30922705